# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [74]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

r = requests.get(url)

r.status_code

200

In [75]:
# 2) Seu código aqui

data_json = r.json()


In [155]:
df = pd.DataFrame(data_json['items'])
df.head(5)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,34100,Município,Resende,3304201,RJ,PVL02.003091/2019-33,Deferido (PVL-IF),None,None,Operação contratual interna,Aporte de capital,Instituição Financeira Nacional,Caixa Econômica Federal,Real,3600000.00,0,1,27/09/2019
1,5713,Município,Volta Redonda,3306305,RJ,None,Deferido,17944.000179/2014-51,2014-03-14T03:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2851305.24,0,0,24/03/2014
2,60039,Município,Petrópolis,3303906,RJ,PVL02.005942/2022-88,Deferido,17944.102384/2022-61,2022-06-21T13:12:44Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,20000000.00,0,1,23/06/2022
3,62993,Município,Paraíba do Sul,3303708,RJ,PVL02.000025/2023-98,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3900000.00,0,1,28/03/2023
4,60354,Município,Paraty,3303807,RJ,PVL02.006783/2022-39,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado do Rio de Janeiro,Real,4794056.75,0,1,28/07/2022


In [77]:
status_frequencia = df['status'].value_counts().head(3)
status_frequencia

status
Deferido        106
Arquivado        53
Regularizado     26
Name: count, dtype: int64

In [131]:
# 3) Seu código aqui

df['ano_status'] = df['data_status'].str.split('/').str[0]

df['ano_status'] = df['data_status'].str.slice(6, 10)

df[['data_status', 'ano_status']].head(10)


,data_status,ano_status
0,27/09/2019,2019
1,24/03/2014,2014
2,23/06/2022,2022
3,28/03/2023,2023
4,28/07/2022,2022
5,28/07/2023,2023
6,29/05/2014,2014
7,17/09/2013,2013
8,22/07/2011,2011
9,25/03/2015,2015


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [132]:
#1) Seu código aqui

url_estado_rj = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
url_minucipio_rj = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

def consulta (UF, url):
    UF = UF.upper()
    lista_uf = states = ['RJ', 'SP']
    if UF in lista_uf:
        r = requests.get(url)
        r.status_code
        data_estado = r.json()
        df_UF = pd.DataFrame(data_estado['items'])
        return df_UF
    else:
        return ('Digite uma informação válida')

df_estado_RJ = consulta('RJ', url_estado_rj)
df_estado_RJ.head(5)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,7080,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000492/2012-28,2012-04-16T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,300000000.0,1,0,03/09/2012
1,8556,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000797/2010-78,2011-06-30T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,112000000.0,1,0,04/07/2011
2,9750,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.001046/2006-92,2006-07-07T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,305737280.0,1,0,13/07/2006
3,10002,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001097/2009-67,2009-09-09T03:00:00Z,Operação contratual externa (com garantia da U...,Profisco,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,19759050.0,1,0,28/09/2009
4,11306,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001408/2009-98,2009-11-23T02:00:00Z,Operação contratual externa (com garantia da U...,Fortalecimento Institucional,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,485000000.0,1,0,25/11/2009


In [133]:
# 2) Seu código aqui
url_estado_mg = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado'

r_mg = requests.get(url_estado_mg)
r_mg.status_code
data_json_mg = r_mg.json()
df_mg = pd.DataFrame(data_json_mg['items'])
df_mg.head(5)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,20458,Estado,Minas Gerais,31,MG,00000.000000/2020-68,Arquivado a pedido,17944.001085/2015-81,2015-11-23T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,5.000000e+07,0,0,29/04/2016
1,13695,Estado,Minas Gerais,31,MG,None,Arquivado,19405.000053/2003-57,2003-08-25T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,3.500000e+07,0,0,20/02/2004
2,13587,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000031/2004-78,2004-11-27T02:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento rural,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,3.500000e+07,1,0,09/12/2004
3,13672,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000048/2005-14,2005-12-28T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.700000e+08,1,0,19/04/2006
4,10011,Estado,Minas Gerais,31,MG,None,Deferido,17944.001099/2010-90,2010-08-23T03:00:00Z,Operação contratual interna,PMAE,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,4.674243e+06,1,0,25/08/2010


In [134]:
solicitacoes_arquivadas_mg = df_mg[(df_mg['status'] == 'Arquivado por decurso de prazo')]

qtd_solicitacoes_arquivadas_mg = solicitacoes_arquivadas_mg.shape[0]
qtd_solicitacoes_arquivadas_mg


1

In [150]:
# 3) Seu código aqui
url_municipio_ba = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Munic%C3%ADpio'
r_ba = requests.get(url_municipio_ba)
r_ba.status_code
data_json_ba = r_ba.json()
df_ba = pd.DataFrame(data_json_ba['items'])
df_ba.head(5)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,31652,Município,Luís Eduardo Magalhães,2919553,BA,PVL02.000780/2019-96,Deferido,17944.101567/2019-63,2019-05-02T17:04:37Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,25403041.71,0,0,08/05/2019
1,18635,Município,Mairi,2920106,BA,None,Deferido,19407.000074/2003-52,2003-12-31T02:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,500000.00,0,0,23/01/2004
2,34231,Município,Sobradinho,2930774,BA,PVL02.003000/2019-60,Deferido,17944.102908/2019-18,2019-08-13T21:15:47Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,22000000.00,0,1,23/08/2019
3,32239,Município,Aramari,2902203,BA,PVL02.001388/2019-64,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,1600000.00,0,0,29/05/2019
4,29928,Município,Santo Antônio de Jesus,2928703,BA,PVL02.002401/2018-11,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,3500000.00,1,1,01/11/2018


In [149]:
solicitacoes_deferidas = df_ba[df_ba['status'] == 'Deferido']

municipio_com_mais_deferidos = solicitacoes_deferidas['interessado'].value_counts().idxmax()
qtd_solicitacoes = solicitacoes_deferidas['interessado'].value_counts().max()

print(f'O município da Bahia com mais solicitações deferidas é {municipio_com_mais_deferidos} com {qtd_solicitacoes} solicitações.')


O município da Bahia com mais solicitações deferidas é Luís Eduardo Magalhães com 16 solicitações.


In [152]:
# 4) Seu código aqui
url_estado_br = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Estado'
r_ba_estado = requests.get(url_estado_br)
r_ba_estado.status_code
data_json_ba_estado = r_ba_estado.json()
df_ba_estado = pd.DataFrame(data_json_ba_estado['items'])
df_ba_estado.head(5)



,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,18633,Estado,Bahia,29,BA,None,Deferido,19407.000073/2003-16,2004-01-30T02:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.150132e+08,1,0,18/03/2004
1,20438,Estado,Bahia,29,BA,00000.000000/2017-83,Deferido,17944.001076/2015-90,2016-03-02T18:35:28Z,Operação contratual interna,Pró-Transporte,Instituição Financeira Nacional,Caixa Econômica Federal,Real,8.000000e+08,1,0,31/03/2016
2,18589,Estado,Bahia,29,BA,None,Deferido,19407.000058/2004-41,2005-11-11T02:00:00Z,Operação contratual interna,Multissetorial,Instituição Financeira Nacional,Banco do Nordeste do Brasil S/A,Dólar dos EUA,1.400000e+07,1,0,08/12/2005
3,18595,Estado,Bahia,29,BA,None,Deferido,19407.000060/2002-58,2002-06-21T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.400000e+06,1,0,03/07/2002
4,18531,Estado,Bahia,29,BA,None,Deferido,19407.000044/2002-65,2002-06-11T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,7.360877e+06,1,0,05/07/2002


In [153]:
df_ba_estado.to_csv('df_ba_estado.csv')